In [6]:
import torch
import torch.nn as nn

In [7]:
device = torch.device('mps')

In [8]:
class DiscriminatorNHL(nn.Module):
    def __init__(self):
        super(DiscriminatorNHL, self).__init__()
        self.ndf = 16  # Size of feature maps in discriminator
        self.nc = 3  # Number of channels of the training images

        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=self.ndf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf, out_channels=self.ndf*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*2, out_channels=self.ndf*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*4, out_channels=self.ndf*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*8, out_channels=self.ndf*16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*16, out_channels=self.ndf*32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 32),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*32, out_channels=self.ndf*64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*64, out_channels=self.ndf*128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*128, out_channels=1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.network(input)

In [9]:
class GeneratorNHL(nn.Module):
    def __init__(self):
        super(GeneratorNHL, self).__init__()

        self.nz = 100  # Size of z latent vector
        self.ngf = 16  # Size of feature maps in generator
        self.nc = 3  # Number of channels in the training images

        self.network = nn.Sequential(
            nn.ConvTranspose2d(in_channels=self.nz, out_channels=self.ngf*128, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(self.ngf*128),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*128, out_channels=self.ngf*64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*64),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*64, out_channels=self.ngf*32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*32),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*32, out_channels=self.ngf*16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*16),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*16, out_channels=self.ngf*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*8, out_channels=self.ngf*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*4, out_channels=self.ngf*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*2, out_channels=self.ngf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf, out_channels=self.nc, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.network(input)

In [10]:
discriminator = DiscriminatorNHL().to(device)
generator = GeneratorNHL().to(device)

In [12]:
real = torch.zeros(1, 3, 1024, 1024)
real = real.to(device)

noise = torch.randn(1, 100, 1, 1)
noise = noise.to(device)
fake = generator(noise)

In [13]:
pred_real = discriminator(real)
pred_fake = discriminator(fake)

print('Pred real: ', pred_real)
print('Pred fake: ', pred_fake)

Pred real:  tensor([[[[0.5000]]]], device='mps:0', grad_fn=<SigmoidBackward0>)
Pred fake:  tensor([[[[0.6012]]]], device='mps:0', grad_fn=<SigmoidBackward0>)
